In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [2]:
! pip install langchain langchain-community langchain-neo4j

In [3]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)


In [5]:
## Dataset Movies
movies_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [6]:
graph.query(movies_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {POB: STRING, name: STRING, YOB: INTEGER}
Company {name: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [8]:
groq_api_key=os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10cf568a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10cf571a0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from langchain_neo4j import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True, allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x105e55880>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10cf568a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10cf571a0>, model_name='Gemma2-9b-It', model_kwa

In [13]:
response=chain.invoke({"query":"Who was the director of the movie Taxi Driver?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Taxi Driver"})<-[:DIRECTED]-(p:Person) RETURN p.name
Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Taxi Driver?',
 'result': 'Martin Scorsese  \n'}

In [15]:
response=chain.invoke({"query":"How many movies has Tim Robbins acted in"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tim Robbins"})-[r:ACTED_IN]->(m:Movie)
RETURN count(r)
Full Context:
[{'count(r)': 2}]

> Finished chain.


{'query': 'How many movies has Tim Robbins acted in',
 'result': '2 movies.  \n'}

In [16]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)
RETURN count(p) AS count
Full Context:
[{'count': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists. \n'}

In [17]:
response=chain.invoke({"query":"How many unique artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN DISTINCT p

Full Context:
[{'p': {'name': 'John Lasseter'}}, {'p': {'name': 'Joe Johnston'}}, {'p': {'name': 'Howard Deutch'}}, {'p': {'name': 'Forest Whitaker'}}, {'p': {'name': 'Charles Shyer'}}, {'p': {'name': 'Michael Mann'}}, {'p': {'name': 'Sydney Pollack'}}, {'p': {'name': 'Peter Hewitt'}}, {'p': {'name': 'Peter Hyams'}}, {'p': {'name': 'Martin Campbell'}}]

> Finished chain.


{'query': 'How many unique artists are there?',
 'result': 'There are 10 unique artists. \n'}

---------